<a href="https://colab.research.google.com/github/wcreis/MyAssistentGeminiAI/blob/main/AssistenteFocadoComGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install -q -U google-generativeai

C
C.utf8
en_US.utf8
POSIX


Imports iniciais e Configuração dos parametros para o sistema funcionar

In [30]:
import google.generativeai as genai
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata
from datetime import datetime
import locale

# Configurar a API do Google Generative AI
GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
# Caso o usuario queira digitar aqui diretamente
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Digite sua API Key do Google Generative AI: ")
genai.configure(api_key=GOOGLE_API_KEY)

# Definir o modelo para embeddings
model = "models/embedding-001"

# Definir o modelo para as Formatações
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
}

model_format = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)


In [51]:
# Função para gerar embeddings
def gerar_embedding(texto):
  """Gera um embedding para o texto."""
  return genai.embed_content(model=model, content=texto, task_type="RETRIEVAL_DOCUMENT")["embedding"]

# Carregar tarefas do arquivo JSON
def carregar_tarefas():
  """Carrega as tarefas e seus embeddings."""
  try:
    with open("tarefas.json", "r") as f:
      data = json.load(f)
    df = pd.DataFrame(data, columns=["id", "summary","description","start","end", "priority", "embedding"])
    df['embedding'] = df['embedding'].apply(np.array)  # Converter strings de embeddings para arrays NumPy
    return df
  except FileNotFoundError:
    return pd.DataFrame(columns=['id', 'summary', 'description', 'start', 'end', 'priority', 'embedding'])

# Salvar tarefas no arquivo JSON
def salvar_tarefas(df):
  """Saves the tasks and their embeddings."""
  data = df.to_dict(orient='records')
  for row in data:
    if isinstance(row['embedding'], np.ndarray):
      row['embedding'] = row['embedding'].tolist()  # Convert NumPy arrays to lists
    elif isinstance(row['embedding'], float):
      row['embedding'] = [row['embedding']]  # Convert float values to single-element lists
    else:
      row['embedding'] = list(row['embedding'])  # Convert other types to lists
  with open("tarefas.json", "w") as f:
    json.dump(data, f, indent=4)

# Adicionar tarefa
def adicionar_tarefa(df, summary, description, start, end, priority):
  """Adiciona uma nova tarefa ao DataFrame."""
  embedding = gerar_embedding(f"{summary} {description}")
  nova_tarefa = {
    'id': str(int(df['id'].max()) + 1) if not df.empty else '1',
    'summary': summary,
    'description': description,
    'start': start,
    'end': end,
    'priority': priority,
    'embedding': embedding
  }
  df = pd.concat([df, pd.DataFrame([nova_tarefa])], ignore_index=True)
  salvar_tarefas(df)
  print("Tarefa adicionada com sucesso!")
  return df

# Listar tarefas
def listar_tarefas(df):
  """Lista todas as tarefas no DataFrame."""
  if df.empty:
    print("Você ainda não possui tarefas.")
  else:
    for index, row in df.iterrows():
        print("-" * 20)
        print(f"{'ID'}: {row['id']}")
        print(f"{'Tarefa'}: {row['summary']}")
        print(f"{'Descrição'}: {row['description']}")
        if isinstance(row['start'], str):
          print(f"{'Início'}: {row['start']}")
        else:
          print(f"{'Início'}: {row['start']}")
        if isinstance(row['end'], str):
          print(f"{'Término'}: {row['end']}")
        else:
          print(f"{'Término'}: {row['end']}")
        print(f"{'Prioridade'}: {row['priority']}")

# Concluir tarefa (remover do DataFrame)
def concluir_tarefa(df, id_tarefa):
  """Marca uma tarefa como concluída."""
  df = df[df['id'] != id_tarefa]
  salvar_tarefas(df)
  print("Tarefa concluída!")
  return df

def exibir_ajuda():
  """Exibe a lista de comandos disponíveis."""
  print("Comandos disponíveis:")
  print("- adicionar: adiciona uma nova tarefa.")
  print("- listar: lista as tarefas.")
  print("- concluir: marca uma tarefa como concluída.")
  print("- ajuda: exibe esta ajuda.")
  print("- sair: encerra o assistente.")

# Loop principal
def main():
  df = carregar_tarefas()
  while True:
    comando = input("Digite um dos comandos abaixo\n\nadicionar\nlistar\nconcluir\nsair\najuda\n\n")

    if comando == "adicionar":
      summary = input("Resumo da tarefa: ")
      description = input("Descrição da tarefa: ")
      start = input("Data e hora de início (opcional): ")
      end = input("Data e hora de término (opcional): ")
      priority = int(input("Prioridade (1-alta, 2-média, 3-baixa): "))
      df = adicionar_tarefa(df, summary, description, start, end, priority)
    elif comando == "listar":
      listar_tarefas(df)
    elif comando == "concluir":
      id_tarefa = input("Digite o ID da tarefa a ser concluída: ")
      df = concluir_tarefa(df, id_tarefa)
    elif comando == "ajuda":
      exibir_ajuda()
    elif comando == "sair":
      break
    else:
      print("Comando inválido. Digite 'ajuda' para ver os comandos disponíveis.")

if __name__ == "__main__":
  main()

Digite um dos comandos abaixo

adicionar
listar
pesquisar
concluir
sair
ajuda

pesquisar
Digite sua pesquisa: alura
Tarefa mais similar:
--------------------
ID: 1
Tarefa: Reunião de Brainstorming
Descrição: Discutir novas ideias para o projeto X.
Digite um dos comandos abaixo

adicionar
listar
pesquisar
concluir
sair
ajuda

listar
--------------------
ID: 1
Tarefa: Reunião de Brainstorming
Descrição: Discutir novas ideias para o projeto X.
Início: {'dateTime': '2024-03-15T10:00:00-03:00'}
Término: {'dateTime': '2024-03-15T11:00:00-03:00'}
Prioridade: 1
--------------------
ID: 2
Tarefa: Concluir relatório financeiro
Descrição: Finalizar o relatório e enviar para revisão.
Início: {'date': '2024-03-18'}
Término: {'date': '2024-03-18'}
Prioridade: 3
--------------------
ID: 3
Tarefa: Ligar para o dentista
Descrição: Agendar consulta para checkup.
Início: {'dateTime': '2024-03-20T14:00:00-03:00'}
Término: {'dateTime': '2024-03-20T14:30:00-03:00'}
Prioridade: 2
--------------------
ID: 4
T

ValueError: shapes (3,1) and (768,) not aligned: 1 (dim 1) != 768 (dim 0)